**Count image**

In [1]:
p_train= ""
p_test= ""

In [2]:
import os
import os.path as path

def count_images(p):
    
    num_img= 0
    
    for folder in os.listdir(p):
        p_f= p+"/"+folder
        
        if path.isdir(p_f):
            for img in os.listdir(p_f):
                
                num_img+=1
    
    return num_img
            

n_train= count_images(p_train)
n_test= count_images(p_test)

print(f"Number of train images: {n_train}")
print(f"Number of test images: {n_test}")
print(f'Number of total images: {n_train+n_test}')

Number of train images: 237989
Number of test images: 2720
Number of total images: 240709


**Image resolutions**

In [4]:
from PIL import Image
from pprint import pprint


def count_image_res(p):
    
    resolutions= dict()
    
    for folder in os.listdir(p):
        p_f= p+"/"+folder
        
        if path.isdir(p_f):
            for f in os.listdir(p_f):
                    with Image.open(p_f+"/"+f) as img:
                        img_size= img.size
                        
                        if img_size in resolutions.keys():
                            resolutions[img_size]+=1
                        else:
                            resolutions[img_size]=1
    
    return resolutions
    
train_res= count_image_res(p_train)
test_res= count_image_res(p_test)

print("Train image resolutions: ")
pprint(train_res)
print("\nTest image resolutions: ")
pprint(test_res)

Train image resolutions: 
{(768, 405): 14873, (768, 432): 223116}

Test image resolutions: 
{(768, 432): 2720}


In [ ]:
from random import sample
import shutil

test_input= ""
train_input= ""
test_output= ""
train_output= ""

def sample_images(input_folder, output_folder, num_img):
    
    all_images = [f for f in os.listdir(input_folder) if f.endswith('.jpg')]
    
    sampled_images= sample(all_images, num_img)
    
    for img in sampled_images:
        shutil.copy(os.path.join(input_folder, img), os.path.join(output_folder, img))

    print(f"Copied {len(sampled_images)} images to {output_folder}")
    
sample_images(train_input, train_output, 9000)
sample_images(test_input, test_output, 1000)

In [4]:
import os

videomatte240k_unpacked= ""
files= os.listdir(videomatte240k_unpacked)
print(len(files))


2720


In [ ]:
from os import path as path
import os
import shutil
from alive_progress import alive_bar


def unpack_split(split_path, videomatte240k_unpacked, split_name):
    
    #source    
    fgr_path= path.join(split_path, 'fgr')
    pha_path= path.join(split_path, 'pha')
    
    #destination
    dest_fgr_path= path.join(videomatte240k_unpacked, split_name, 'fgr')
    dest_pha_path= path.join(videomatte240k_unpacked, split_name, 'pha')
    n_img= 0
    
    #scenes
    for scene_fgr, scene_pha in zip(os.listdir(fgr_path),os.listdir(pha_path)):
        scene_fgr_path=path.join(fgr_path, scene_fgr)
        scene_pha_path= path.join(pha_path, scene_pha)
        
        #images
        if path.isdir(scene_fgr_path):
            for img_fgr, img_pha in zip(os.listdir(scene_fgr_path), os.listdir(scene_pha_path)):
                            
                name= scene_fgr[-4:]+'_'+img_fgr #provide a unique file_name (scene_img.jpg) 
                
                #copy foreground and alpha matt
                shutil.copy(path.join(scene_fgr_path, img_fgr), path.join(dest_fgr_path, name))            
                shutil.copy(path.join(scene_pha_path, img_pha), path.join(dest_pha_path, name))
                n_img+=2
                                
                
        
    print(f'Copied {n_img} images from {split_path} to {path.join(videomatte240k_unpacked, split_name)}')
        

def unpack_videomatte240k(videomatte240k, videomatte240k_unpacked):
    videomatte_train= path.join(videomatte240k, "train")
    videomatte_test= path.join(videomatte240k, "test")
    
    unpack_split(videomatte_train,videomatte240k_unpacked, 'train')
    unpack_split(videomatte_test,videomatte240k_unpacked, 'test')
    
    

videomatte240k= ""
videomatte240k_unpacked= "" 

unpack_videomatte240k(videomatte240k, videomatte240k_unpacked)

In [9]:
from sklearn.model_selection import train_test_split
import os 
from os import path

def move_splits(split_source: str, split_dest: str, fgrs: list, phas: list):
    for fgr, pha in zip(fgrs, phas):
        fgr_path= path.join(split_source,"fgr", fgr)
        pha_path= path.join(split_source, 'pha', pha)
        shutil.copy(fgr_path, path.join(split_dest, 'fgr'))
        shutil.copy(pha_path, path.join(split_dest, 'pha'))
    
    print(f'Moved {len(fgrs)} images from {split_source} to {split_dest}')    
    
def split_dataset(raw_folder, composite_folder):
    train_path= path.join(raw_folder, 'train')
    
    #split train set in train and val sets (equal random state to keep order)
    train_fgr, val_fgr= train_test_split(
        os.listdir(path.join(train_path, 'fgr')), test_size=3007/237989, random_state=42)
    train_pha, val_pha= train_test_split(
        os.listdir(path.join(train_path, 'pha')), test_size=3007/237989, random_state=42)
    
    move_splits(train_path, path.join(composite_folder, 'training', 'train'), train_fgr, train_pha)
    move_splits(train_path, path.join(composite_folder, 'training', 'val'), val_fgr, val_pha)
    
    #just copy test files to dest folder
    test_path= path.join(raw_folder, 'test')
    test_fgr= os.listdir(path.join(test_path, 'fgr'))
    test_pha= os.listdir(path.join(test_path, "pha"))
    move_splits(test_path, path.join(composite_folder, 'test'), test_fgr, test_pha)
    
    

split_dataset("/Users/jannisschuhmann/Documents/Uni/Semester5/ProjectCVPR/EFormer-Reproduction/data/raw/VideoMatte240k_unpacked",
              "/Users/jannisschuhmann/Documents/Uni/Semester5/ProjectCVPR/EFormer-Reproduction/data/composite_dataset")

In [27]:
print(len(os.listdir('/Users/jannisschuhmann/Documents/Uni/Semester5/ProjectCVPR/EFormer-Reproduction/data/composite_dataset/test/backgrounds')))
print(len(os.listdir('/Users/jannisschuhmann/Documents/Uni/Semester5/ProjectCVPR/EFormer-Reproduction/data/composite_dataset/test/pha')))
print(len(os.listdir('/Users/jannisschuhmann/Documents/Uni/Semester5/ProjectCVPR/EFormer-Reproduction/data/composite_dataset/test/fgr')))
print(len(os.listdir('/Users/jannisschuhmann/Documents/Uni/Semester5/ProjectCVPR/EFormer-Reproduction/data/composite_dataset/training/train/fgr')))
print(len(os.listdir('/Users/jannisschuhmann/Documents/Uni/Semester5/ProjectCVPR/EFormer-Reproduction/data/composite_dataset/training/train/pha')))
print(len(os.listdir('/Users/jannisschuhmann/Documents/Uni/Semester5/ProjectCVPR/EFormer-Reproduction/data/composite_dataset/training/val/fgr')))
print(len(os.listdir('/Users/jannisschuhmann/Documents/Uni/Semester5/ProjectCVPR/EFormer-Reproduction/data/composite_dataset/training/val/pha')))
print(len(os.listdir('/Users/jannisschuhmann/Documents/Uni/Semester5/ProjectCVPR/EFormer-Reproduction/data/composite_dataset/training/backgrounds')))

1000
2720
2720
234982
234982
3007
3007
9000
